In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [78]:
#put helper functions/calculations here!

def con(x):
    """Convert to float if possible"""
    try:
        return float(x)
    except:
        return None

In [140]:
data = pd.read_csv("Vizier/hip_full_formatted2.csv", sep=",", low_memory=False)
data

,_Glon,_Glat,_RAJ2000,_DEJ2000,_RAB1950,_DEB1950,_Elon2000,_Elat2000,_time,_etime,...,BD,CoD,CPD,(V-I)red,SpType,r_SpType,HIPep,Erratum,_RA.icrs,_DE.icrs
0,111.3597,-36.417,2.5542817,25.5267532,1.9084505,25.2485924,13.0113022,22.2950371,,0,...,B+24 7,,,0.42,F0III,X,HIPep,Erratum,2.55428171,25.526753
1,117.8018,-2.4347,2.5587584,60.0260469,1.8993345,59.7479435,36.181116,51.8039073,,0,...,B+59 4,,,1.23,G8III,X,HIPep,Erratum,2.5587584,60.026047
2,116.328,-11.4583,2.5680176,50.8848982,1.9129509,50.6066901,27.9612575,44.4761435,,0,...,B+50 14,,,1.08,K0,S,HIPep,Erratum,2.56801765,50.884898
3,116.5543,-10.1061,2.5685944,52.2551367,1.9137442,51.9770508,29.0524755,45.6089037,,0,...,B+51 12,,,-0.03,B9,S,HIPep,Erratum,2.56859443,52.255137
4,307.7633,-45.9035,2.5746672,-70.8368948,1.9654422,-71.115109,312.1564164,-60.7514104,,0,...,,,P-71 2,0.48,F2/F3V,1,HIPep,Erratum,2.5746672,-70.836895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118213,305.5981,-38.6129,2.5334041,-78.300717,1.9452866,-78.5786775,297.9814094,-64.4204886,,0,...,,,P-78 4,1.33,K3III,1,HIPep,Erratum,2.53340406,-78.300717
118214,320.0866,-66.5871,2.5389894,-49.1433742,1.909107,-49.4212892,337.2332878,-44.8670259,,0,...,,C-49 20,P-49 15,0.56,F5V,2,HIPep,Erratum,2.53898941,-49.143374
118215,355.806,-79.1764,2.5423143,-33.2810479,1.9084617,-33.5587074,347.5585568,-31.2126153,,0,...,,C-33 26,P-33 10,0.83,G8V,3,HIPep,Erratum,2.54231429,-33.281048
118216,67.8416,-78.1572,2.5433657,-19.9835787,1.9063664,-20.2616779,354.0603912,-19.2772135,,0,...,B-20 9,,,0.60,F7V,4,HIPep,Erratum,2.5433657,-19.983579


In [116]:
data = data.convert_dtypes()
data["Plx"] = data["Plx"].apply(con)
data["e_Plx"] = data["e_Plx"].apply(con)
data.columns

Index(['_Glon', '_Glat', '_RAJ2000', '_DEJ2000', '_RAB1950', '_DEB1950',
       '_Elon2000', '_Elat2000', '_time', '_etime', 'HIP', 'Proxy', 'RAhms',
       'DEdms', 'Vmag', 'VarFlag', 'r_Vmag', 'RAICRS', 'DEICRS', 'AstroRef',
       'Plx', 'pmRA', 'pmDE', 'e_RAICRS', 'e_DEICRS', 'e_Plx', 'e_pmRA',
       'e_pmDE', 'DE:RA', 'Plx:RA', 'Plx:DE', 'pmRA:RA', 'pmRA:DE', 'pmRA:Plx',
       'pmDE:RA', 'pmDE:DE', 'pmDE:Plx', 'pmDE:pmRA', 'F1', 'F2', 'BTmag',
       'e_BTmag', 'VTmag', 'e_VTmag', 'm_BTmag', 'B-V', 'e_B-V', 'r_B-V',
       'V-I', 'e_V-I', 'r_V-I', 'CombMag', 'Hpmag', 'e_Hpmag', 'Hpscat',
       'o_Hpmag', 'm_Hpmag', 'Hpmax', 'HPmin', 'Period', 'HvarType', 'moreVar',
       'morePhoto', 'CCDM', 'n_CCDM', 'Nsys', 'Ncomp', 'MultFlag', 'Source',
       'Qual', 'm_HIP', 'theta', 'rho', 'e_rho', 'dHp', 'e_dHp', 'Survey',
       'Chart', 'Notes', 'HD', 'BD', 'CoD', 'CPD', '(V-I)red', 'SpType',
       'r_SpType', 'HIPep', 'Erratum', '_RA.icrs', '_DE.icrs', 'Dist'],
      dtype='object')

In [117]:
#parallax is given in milliarcseconds (mas). Following is an ordered list (same order as columns) of units:
#deg|deg|deg|deg|deg|deg|deg|deg|d|s| | | | |mag| | |deg|deg| |mas|mas/yr|mas/yr|mas|mas|mas|mas/yr|mas/yr| | | | | | | | | | |%| |mag|mag|mag|mag| |mag|mag| |mag|mag| | |mag|mag|mag| | |mag|mag|d| | | | | | | | | | | |deg|arcsec|arcsec|mag|mag| | | | | | | |mag| | | | |deg|deg


In [134]:
data["Dist"] = abs((1/data["Plx"])*1000) #convert mas to parsecs
data["e_Dist_pos"] = ((1/(data["Plx"]-data["e_Plx"]))*1000) - data["Dist"]
data["e_Dist_neg"] = ((1/(data["Plx"]+data["e_Plx"]))*1000) - data["Dist"]

In [119]:
data[data["Plx"] == max(data["Plx"])]["Plx"]

69992    772.33
Name: Plx, dtype: float64

In [137]:
#get all stars that are within 50pc (include stars where Dist-|e_Dist_neg| < 50, i.e. where the standard error makes it possible for the star to be within 50 pc)
data[(data["Dist"]+data["e_Dist_neg"] < 50.0)][["HIP","Dist", "e_Dist_neg"]].sort_values("Dist", ascending = False)

,HIP,Dist,e_Dist_neg
63397,64289,16666.666667,-16624.257846
16134,16983,3571.428571,-3532.803620
38803,39670,2702.702703,-2666.731480
44422,45292,2325.581395,-2283.932083
108275,109205,1818.181818,-35151.515152
...,...,...,...
53161,54035,2.548420,-0.005896
87024,87937,1.821460,-0.005227
70784,71683,1.347491,-0.002537
70781,71681,1.347491,-0.002537


In [141]:
data[data["HIP"] == 64289][["Plx","e_Plx"]]

,Plx,e_Plx
63397,-0.06,23.64


In [124]:
data[["Plx","e_Plx","e_Dist","Dist"]]

,Plx,e_Plx,e_Dist,Dist
0,3.93,1.13,884.955752,254.452926
1,1.12,0.83,1204.819277,892.857143
2,2.74,1.18,847.457627,364.963504
3,2.98,0.81,1234.567901,335.570470
4,5.26,0.97,1030.927835,190.114068
...,...,...,...,...
118213,3.21,0.60,1666.666667,311.526480
118214,8.50,2.90,344.827586,117.647059
118215,-2.57,1.52,657.894737,389.105058
118216,6.10,1.56,641.025641,163.934426


In [133]:
temp = (1/(3.93))*1000
(1/(3.93+1.13))*1000-temp

-56.824467710627715

In [132]:
(1/(3.93-1.13))*1000 - temp

102.6899309342057